In [1]:
import nltk
import re
import codecs
import csv
import os
from sklearn import metrics

path = 'D:\Dropbox\ongoing\chrisIntervention'

In [2]:
# read cohort info from the CSV and save them with the sbj ID

intvIdx = 'Ex2'

category = {}
with open(os.path.join(path, intvIdx + '_key.csv')) as f:
    sbjData = csv.reader(f)
    next(sbjData) # skip the header
    for row in sbjData:
        category[row[0]] = int(row[1])

subjectId = category.keys()        
        
#print(subjectId)
#print()
print('# of cohort 1: ', sum(1 for sbj in category if category[sbj] == 1))
print('# of cohort 2: ', sum(1 for sbj in category if category[sbj] == 2))


# of cohort 1:  149
# of cohort 2:  213


In [3]:
# build the dictionary based on the training set
# this is just to check if it is working
# we should build the dictionary for each leave-one-out test

def make_dictionary(subjectPool, critNumSbj):
    allWords = []
    for sbj in subjectPool:
        sbjWords = []
        for line in codecs.open(os.path.join(path, intvIdx, sbj + '.txt'), 'r' , 'utf-8'):
            line = line.strip()
            line = line.lower()
            tokens = nltk.wordpunct_tokenize(line)
            text = nltk.Text(tokens)
            words = [w for w in text if re.search('[a-z]+',w) and len(w)>2]
            sbjWords.extend(words)
        # when making dictionary, make the words from a document unique
        sbjWords = list(set(sbjWords))
        allWords.extend(sbjWords)
    
    # once we make list of all words, we count the words
    cntWords = nltk.FreqDist(allWords)
    #for ii in range(21,0,-2):
    #    print('# words with freq >', ii, ' : ', sum(1 for word in cntWords if cntWords[word] > ii))
    
    # we go with the words that came from at least N different people (arbitrary) 
    # we can change and see what happens ...
    return [word for word in cntWords if cntWords[word] > critNumSbj]

dictionary = make_dictionary(subjectId, 5)

print(dictionary)

['always', 'problems', 'sad', 'play', 'then', 'importent', 'for', 'great', 'after', 'shows', 'funny', 'support', 'started', 'picked', 'who', 'import', 'them', 'geting', 'humor', 'ever', 'down', 'listening', 'live', 'most', 'art', 'these', 'spending', 'basketball', 'doing', 'was', 'draw', 'work', 'things', 'lot', 'close', 'last', 'sports', 'express', 'through', 'almost', 'home', 'listen', 'mad', 'want', 'your', 'group', 'one', 'favorite', 'back', 'they', 'learn', 'important', 'wanna', 'being', 'values', 'happy', 'would', 'become', 'proud', 'cool', 'show', 'why', 'active', 'years', 'other', 'family', 'won', 'sometimes', 'any', 'singing', 'gets', 'what', 'never', 'mean', 'while', '6th', 'will', 'this', 'come', 'believe', 'thing', 'drawing', 'dont', 'use', 'calms', 'college', 'smile', 'might', 'keep', 'people', 'free', 'everyone', 'sing', 'future', 'someone', 'spend', 'football', 'something', 'make', 'god', 'when', 'know', 'depend', 'making', 'little', 'alone', 'their', 'reason', 'chose', 

In [4]:
# we define a function that tunrs text into a list of tokens

def extract_feature(text, dictionary):
    text = text.strip()
    text = text.lower()
    tokens = nltk.wordpunct_tokenize(text)
    text = nltk.Text(tokens)
    feature = {}
    for word in dictionary:
        feature[word] = (word in text)
    return feature

print('nltk output: ', nltk.wordpunct_tokenize('You think you can do this one, Kate?'))
print(extract_feature('You think you can do this one, Kate?', dictionary))

nltk output:  ['You', 'think', 'you', 'can', 'do', 'this', 'one', ',', 'Kate', '?']
{'will': False, 'there': False, 'enjoy': False, 'this': True, 'lot': False, 'close': False, 'outside': False, 'believe': False, 'sence': False, 'you': True, 'self': False, 'need': False, 'help': False, 'come': False, 'smart': False, 'life': False, 'into': False, 'always': False, 'hang': False, 'social': False, 'anything': False, 'last': False, 'dance': False, 'team': False, 'how': False, 'problems': False, 'way': False, 'like': False, 'sports': False, 'cause': False, 'day': False, 'put': False, 'sometimes': False, 'job': False, 'also': False, 'new': False, 'thing': False, 'basketball': False, 'express': False, 'wont': False, 'chose': False, 'through': False, 'around': False, 'where': False, 'means': False, 'almost': False, 'know': False, 'play': False, 'then': False, 'home': False, 'more': False, 'ever': False, 'importent': False, 'grow': False, 'things': False, 'listen': False, 'mad': False, 'soccer': 

In [5]:
# leave-one-out classification

predLabel = []
trueLabel = []
currSbj = 1;

for sbj in subjectId:
    # sbj is being tested, others are used to train the classifier
    trainSbj = list(subjectId)
    trainSbj.remove(sbj)
    # create the dictionary
    trainDict = make_dictionary(trainSbj, 5)
    trainText = []
    # create the training set
    for tSbj in trainSbj:
        tmpText = ''
        for line in codecs.open(os.path.join(path, intvIdx, tSbj + '.txt'), 'r' , 'utf-8'):
            tmpText = tmpText + ' ' + line
        trainText.append([extract_feature(tmpText, trainDict), category[tSbj]])
    # train classifier
    classifier = nltk.NaiveBayesClassifier.train(trainText)

    # now the test set
    trueLabel.append( category[sbj] )
    testText = ''
    for line in codecs.open(os.path.join(path, intvIdx, sbj + '.txt'), 'r' , 'utf-8'):
        testText = testText + ' ' + line
    # predict!
    currPred = classifier.classify(extract_feature(testText, trainDict))
    predLabel.append( currPred )
    
    print(str(currSbj) + ' (' + sbj + '): pred - ' + str(currPred) + ', actual - ' + str(category[sbj]) )
    currSbj = currSbj + 1

print('done. ')


1 (320075_2): pred - 2, actual - 2
2 (520145_2): pred - 2, actual - 2
3 (720059_2): pred - 2, actual - 2
4 (220131_2): pred - 2, actual - 2
5 (1020119_2): pred - 2, actual - 2
6 (710608_2): pred - 1, actual - 1
7 (1110116_2): pred - 1, actual - 1
8 (1020070_2): pred - 1, actual - 2
9 (120026_2): pred - 1, actual - 2
10 (420066_2): pred - 1, actual - 2
11 (920004_2): pred - 1, actual - 2
12 (610713_2): pred - 1, actual - 1
13 (220040_2): pred - 2, actual - 2
14 (1120117_2): pred - 1, actual - 2
15 (212202_2): pred - 1, actual - 1
16 (210905_2): pred - 1, actual - 1
17 (1110609_2): pred - 2, actual - 1
18 (920016_2): pred - 2, actual - 2
19 (120081_2): pred - 2, actual - 2
20 (120086_2): pred - 2, actual - 2
21 (620169_2): pred - 2, actual - 2
22 (320054_2): pred - 2, actual - 2
23 (610114_2): pred - 1, actual - 1
24 (110017_2): pred - 2, actual - 1
25 (810903_2): pred - 2, actual - 1
26 (120045_2): pred - 2, actual - 2
27 (320037_2): pred - 2, actual - 2
28 (320069_2): pred - 2, actual 

In [6]:
# summarize the results

print(metrics.classification_report(trueLabel, predLabel))


             precision    recall  f1-score   support

          1       0.71      0.62      0.66       149
          2       0.75      0.82      0.79       213

avg / total       0.74      0.74      0.73       362



In [7]:
# ok, it is POSSIBLE TO PREDICT cohort from the writings.
# now, show me the informative features

dictionary = make_dictionary(subjectId, 5)
print(len(dictionary))

allText = []

for sbj in subjectId:
    tmpText = ''
    for line in codecs.open(os.path.join(path, intvIdx, sbj + '.txt'), 'r' , 'utf-8'):
        tmpText = tmpText + ' ' + line
        allText.append([extract_feature(tmpText, dictionary), category[sbj]])

# train classifier based on all information
classifier = nltk.NaiveBayesClassifier.train(allText)


274


In [8]:
classifier.show_most_informative_features(100)


Most Informative Features
                    from = True                1 : 2      =      7.7 : 1.0
                  values = True                1 : 2      =      7.7 : 1.0
                    stay = True                1 : 2      =      7.1 : 1.0
                 parents = True                1 : 2      =      6.2 : 1.0
                   stuff = True                1 : 2      =      5.5 : 1.0
                   house = True                1 : 2      =      5.2 : 1.0
                     two = True                1 : 2      =      5.2 : 1.0
                  helped = True                1 : 2      =      5.2 : 1.0
                    give = True                1 : 2      =      5.2 : 1.0
                   learn = True                1 : 2      =      5.1 : 1.0
                    take = True                1 : 2      =      4.9 : 1.0
                    been = True                1 : 2      =      4.6 : 1.0
                    team = True                1 : 2      =      4.3 : 1.0